In [1]:
import simpy

## Parallel Processes

In [2]:
def clock(env,name,tick):
    while True:
        print(name,env.now)
        yield env.timeout(tick)

In [3]:
env = simpy.Environment()
env.process(clock(env,'fast',0.5))
env.process(clock(env,'slow',1))
env.run(until=2.1)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5
slow 2
fast 2.0


## Return Value (no break)

In [4]:
def example(env):
    value = yield env.timeout(1,value=42)
    print('now=%d, value=%d' % (env.now,value))

In [7]:
env = simpy.Environment()
p = env.process(example(env))
env.run()

now=1, value=42


## Return Value (with break)

In [8]:
def my_proc(env):
    yield env.timeout(1)
    return 'Armen'

In [9]:
env = simpy.Environment()
proc = env.process(my_proc(env))
env.run(until=proc)

'Armen'

## Step by Step

In [10]:
def my_proc(env):
    while True:
        yield env.timeout(1)
        print('Armen at %d' % env.now)

In [11]:
env = simpy.Environment()
env.process(my_proc(env))
until = 10
while env.peek() < until:
    env.step()

Armen at 1
Armen at 2
Armen at 3
Armen at 4
Armen at 5
Armen at 6
Armen at 7
Armen at 8
Armen at 9


## Process monitoring

In [14]:
def subfunc(env):
    print(env.active_process)

def my_proc(env):
    while True:
        print(env.active_process)
        subfunc(env)
        yield env.timeout(1)

In [15]:
env = simpy.Environment()
p1 = env.process(my_proc(env))
print(env.active_process)
env.step()
print(env.active_process)

None
<Process(my_proc) object at 0x25bf3ea72c8>
<Process(my_proc) object at 0x25bf3ea72c8>
None


## Sleep until woken up

In [16]:
from random import seed, randint
seed(23)

class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))
        self.bat_ctrl_proc = env.process(self.bat_ctrl(env))
        self.bat_ctrl_reactivate = env.event()

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1–6 hours
            print('Start parking at', env.now)
            self.bat_ctrl_reactivate.succeed()  # "reactivate"
            self.bat_ctrl_reactivate = env.event()
            yield env.timeout(randint(60, 360))
            print('Stop parking at', env.now)

    def bat_ctrl(self, env):
        while True:
            print('Bat. ctrl. passivating at', env.now)
            yield self.bat_ctrl_reactivate  # "passivate"
            print('Bat. ctrl. reactivated at', env.now)

            # Intelligent charging behavior here …
            yield env.timeout(randint(30, 90))

In [17]:
env = simpy.Environment()
ev = EV(env)
env.run(until=150)

Bat. ctrl. passivating at 0
Start parking at 29
Bat. ctrl. reactivated at 29
Bat. ctrl. passivating at 60
Stop parking at 131


## Waiting for another process to terminate

In [22]:
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1–6 hours
            print('Start parking at', env.now)
            charging = env.process(self.bat_ctrl(env))
            parking = env.timeout(randint(60, 360))
            yield charging & parking
            print('Stop parking at', env.now)

    def bat_ctrl(self, env):
        print('Bat. ctrl. started at', env.now)
        # Intelligent charging behavior here …
        yield env.timeout(randint(30, 90))
        print('Bat. ctrl. done at', env.now)

In [25]:
env = simpy.Environment()
ev = EV(env)
env.run(until=310)

Start parking at 21
Bat. ctrl. started at 21
Bat. ctrl. done at 101
Stop parking at 186
Start parking at 207
Bat. ctrl. started at 207
Bat. ctrl. done at 279


## Interrupting another process

In [40]:
class EV:
    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive(env))

    def drive(self, env):
        while True:
            # Drive for 20-40 min
            yield env.timeout(randint(20, 40))

            # Park for 1 hour
            print('Start parking at', env.now)
            charging = env.process(self.bat_ctrl(env))
            parking = env.timeout(60)
            yield charging | parking
            if not charging.triggered:
                # Interrupt charging if not already done.
                charging.interrupt('Need to go!')
            print('Stop parking at', env.now)

    def bat_ctrl(self, env):
        print('Bat. ctrl. started at', env.now)
        try:
            yield env.timeout(randint(60, 90))
            print('Bat. ctrl. done at', env.now)
        except simpy.Interrupt as i:
            # Onoes! Got interrupted before the charging was done.
            print('Bat. ctrl. interrupted at', env.now, 'msg:', i.cause)

In [41]:
env = simpy.Environment()
ev = EV(env)
env.run(until=100)

Start parking at 21
Bat. ctrl. started at 21
Stop parking at 81
Bat. ctrl. interrupted at 81 msg: Need to go!


## Resource

In [45]:
def resource_user(env, resource):
    request = resource.request()  # Generate a request event
    yield request                 # Wait for access
    print('Resource request', env.now)
    yield env.timeout(1)          # Do something
    resource.release(request)     # Release the resource
    print('Resource release', env.now)

In [46]:
env = simpy.Environment()
res = simpy.Resource(env, capacity=1)
user = env.process(resource_user(env, res))
env.run()

Resource request 0
Resource release 1


In [47]:
def resource_user(env, resource):
    with resource.request() as req:  # Generate a request event
        yield req                    # Wait for access
        print('Resource request', env.now)
        yield env.timeout(1)         # Do something
                                     # Resource released automatically
        print('Resource release', env.now)

In [49]:
env = simpy.Environment()
res = simpy.Resource(env, capacity=1)
user = env.process(resource_user(env, res))
env.run()

Resource request 0
Resource release 1


## Resource Stats

In [50]:
def print_stats(res):
    print(f'{res.count} of {res.capacity} slots are allocated.')
    print(f'  Users: {res.users}')
    print(f'  Queued events: {res.queue}')


def user(res):
    print_stats(res)
    with res.request() as req:
        yield req
        print_stats(res)
    print_stats(res)

In [51]:
env = simpy.Environment()
res = simpy.Resource(env, capacity=1)
procs = [env.process(user(res)), env.process(user(res))]
env.run()

0 of 1 slots are allocated.
  Users: []
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x25bf3e15e48>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x25bf3e15e48>]
  Queued events: [<Request() object at 0x25bf3e15348>]
0 of 1 slots are allocated.
  Users: []
  Queued events: [<Request() object at 0x25bf3e15348>]
1 of 1 slots are allocated.
  Users: [<Request() object at 0x25bf3e15348>]
  Queued events: []
0 of 1 slots are allocated.
  Users: []
  Queued events: []


## Priority Resource

In [58]:
def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print(f'{name} requesting at {env.now} with priority={prio}')
        yield req
        print(f'{name} got resource at {env.now}')
        yield env.timeout(3)

In [59]:
env = simpy.Environment()
res = simpy.PriorityResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
3 got resource at 3
2 got resource at 6


## Preemptive Resource

In [56]:
def resource_user(name, env, resource, wait, prio):
    yield env.timeout(wait)
    with resource.request(priority=prio) as req:
        print(f'{name} requesting at {env.now} with priority={prio}')
        yield req
        print(f'{name} got resource at {env.now}')
        try:
            yield env.timeout(3)
        except simpy.Interrupt as interrupt:
            by = interrupt.cause.by
            usage = env.now - interrupt.cause.usage_since
            print(f'{name} got preempted by {by} at {env.now}'
                f' after {usage}')

In [57]:
env = simpy.Environment()
res = simpy.PreemptiveResource(env, capacity=1)
p1 = env.process(resource_user(1, env, res, wait=0, prio=0))
p2 = env.process(resource_user(2, env, res, wait=1, prio=0))
p3 = env.process(resource_user(3, env, res, wait=2, prio=-1))
env.run()

1 requesting at 0 with priority=0
1 got resource at 0
2 requesting at 1 with priority=0
3 requesting at 2 with priority=-1
1 got preempted by <Process(resource_user) object at 0x25bf3dc4e88> at 2 after 2
3 got resource at 2
2 got resource at 5


## Preemptive vs Priority

In [60]:
def user(name, env, res, prio, preempt):
    with res.request(priority=prio, preempt=preempt) as req:
        try:
            print(f'{name} requesting at {env.now}')
            assert isinstance(env.now, int), type(env.now)
            yield req
            assert isinstance(env.now, int), type(env.now)
            print(f'{name} got resource at {env.now}')
            yield env.timeout(3)
        except simpy.Interrupt:
            print(f'{name} got preempted at {env.now}')

In [61]:
env = simpy.Environment()
res = simpy.PreemptiveResource(env, capacity=1)
A = env.process(user('A', env, res, prio=0, preempt=True))
env.run(until=1)  # Give A a head start

A requesting at 0
A got resource at 0


In [62]:
B = env.process(user('B', env, res, prio=-2, preempt=False))
C = env.process(user('C', env, res, prio=-1, preempt=True))
env.run()

B requesting at 1
C requesting at 1
B got resource at 3
C got resource at 6


## Containers

In [63]:
class GasStation:
    def __init__(self, env):
        self.fuel_dispensers = simpy.Resource(env, capacity=2)
        self.gas_tank = simpy.Container(env, init=100, capacity=1000)
        self.mon_proc = env.process(self.monitor_tank(env))

    def monitor_tank(self, env):
        while True:
            if self.gas_tank.level < 100:
                print(f'Calling tanker at {env.now}')
                env.process(tanker(env, self))
            yield env.timeout(15)

def tanker(env, gas_station):
    yield env.timeout(10)  # Need 10 Minutes to arrive
    print(f'Tanker arriving at {env.now}')
    amount = gas_station.gas_tank.capacity - gas_station.gas_tank.level
    yield gas_station.gas_tank.put(amount)


def car(name, env, gas_station):
    print(f'Car {name} arriving at {env.now}')
    with gas_station.fuel_dispensers.request() as req:
        yield req
        print(f'Car {name} starts refueling at {env.now}')
        yield gas_station.gas_tank.get(40)
        yield env.timeout(5)
        print(f'Car {name} done refueling at {env.now}')

def car_generator(env, gas_station):
    for i in range(4):
        env.process(car(i, env, gas_station))
        yield env.timeout(5)

In [64]:
env = simpy.Environment()
gas_station = GasStation(env)
car_gen = env.process(car_generator(env, gas_station))
env.run(35)

Car 0 arriving at 0
Car 0 starts refueling at 0
Car 1 arriving at 5
Car 0 done refueling at 5
Car 1 starts refueling at 5
Car 2 arriving at 10
Car 1 done refueling at 10
Car 2 starts refueling at 10
Calling tanker at 15
Car 3 arriving at 15
Car 3 starts refueling at 15
Tanker arriving at 25
Car 2 done refueling at 30
Car 3 done refueling at 30


## Stores

In [65]:
def producer(env, store):
    for i in range(100):
        yield env.timeout(2)
        yield store.put(f'spam {i}')
        print(f'Produced spam at', env.now)

def consumer(name, env, store):
    while True:
        yield env.timeout(1)
        print(name, 'requesting spam at', env.now)
        item = yield store.get()
        print(name, 'got', item, 'at', env.now)

In [66]:
env = simpy.Environment()
store = simpy.Store(env, capacity=2)
prod = env.process(producer(env, store))
consumers = [env.process(consumer(i, env, store)) for i in range(2)]
env.run(until=5)

0 requesting spam at 1
1 requesting spam at 1
Produced spam at 2
0 got spam 0 at 2
0 requesting spam at 3
Produced spam at 4
1 got spam 1 at 4
